In [7]:
import requests
import json
from bs4 import BeautifulSoup
import urllib.request
import base64

In [116]:
import os
from ole import OleFile
import olefile
# from tika import parser
# import PyPDF2
# from pdfminer.pdfinterp import PDFResourceManager, process_pdf
# from pdfminer.converter import TextConverter
# from pdfminer.layout import LAParams
# from io import StringIO
# from io import open
# from urllib.request import urlopen

### ```pip install olefile``` 
olefile 모듈 다운받기
(hwp to text)

### parameter
- seviceKey : id
- target : admrul
- query : 검색어(default: * \)
- numOfRows : 한 페이지 결과수
- pageNo : 페이지 번호

공공데이터 포털API로 행정규칙 id리스트를 얻기 위해 ServiceKey 발급
(https://www.data.go.kr/subMain.jsp?param=T1BFTkFQSUAxNTAwMDExNQ==#/L3B1YnIvcG90L215cC9Jcm9zTXlQYWdlL29wZW5EZXZHdWlkZVBhZ2UkQF4wMTJtMSRAXnB1YmxpY0RhdGFQaz0xNTAwMDExNSRAXnB1YmxpY0RhdGFEZXRhaWxQaz11ZGRpOjI5ZDBkZjIyLWQxOWItNGY2Ny04Y2YyLWI3ZTM3MDQyZDc3Yl8yMDE4MDUxMTE0MzEkQF5vcHJ0aW5TZXFObz0yMDI4NCRAXm1haW5GbGFnPXRydWU=)

In [9]:
ServiceKey ='qHBjb9yik8lx%2B3bJ3tLeWux4VlZOlRcSe0otBb3Yh4T5c24Yia5vI6e%2BVFFUYHRwWl8FgTAvgRJjWf9XiIlpQg%3D%3D'

In [15]:
def get_url(page):
    id_ls = []
    url = 'http://apis.data.go.kr/1170000/law/admrulSearchList.do?'
    url += 'serviceKey=' + ServiceKey + '&query=*&target=admrul&numOfRows=10&pageNo='+ str(page)
    
    res = requests.get(url).text
    soup = BeautifulSoup(res,'html.parser')
    crawl_ls = soup.select('admrul')

    for crawl in crawl_ls:
        crawl = str(crawl)

        start = crawl.index('ID=')
        crawl = crawl[start+3:]
        end = crawl.index('&')
        crawl = crawl[:end]

        id_ls.append(crawl)
    
    return id_ls

In [11]:
def get_file(id_ls):
    params = {}
    url_detail = 'http://www.law.go.kr/LSW/admRulAttFlList.do' 
    file_ls = []
    for admid in id_ls:
        file_info = []
        params.update({'admRulSeq':admid})
        res = requests.post(url_detail, data=params).text
        soup = BeautifulSoup(res,'html.parser')
        files = soup.select('ul li a')
        for file in files:
            file_info.append([file['href'],file.text])
        file_ls.append(file_info)
    
    return file_ls

In [33]:
def download(url, filename):
    if os.path.exists(filename) is False:
        with open(filename,"wb") as file:
            res = requests.get(url)
            file.write(res.content)

In [25]:
def read_pdf_file(pdfFile):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    process_pdf(rsrcmgr, device, pdfFile)
    device.close()

    content = retstr.getvalue()
    retstr.close()
    
    return content

In [256]:
def read_hwp_file(hwpFile):
    path = os.path.realpath(hwpFile)
    f = OleFile(hwpFile)
    for path in f.list_entries(include_storages=False):
        entry = f.get_entry(path)
        name = b'/'.join(x.encode('unicode-escape') for x in path).decode()
        print('%-3d  %-8d  %s' % (entry.id, entry.stream_size, name))
        print(f.get_stream('BodyText/Section1').decode('UTF-16'))
#     with olefile.OleFileIO(hwpFile) as ole:
#         f = ole.openstream('PrvText')
#         encoded_text = f.read()
#         decoded_text = encoded_text.decode('UTF-16')
#         print(decoded_text)

In [257]:
for files in file_ls:
    for file in files:
        print(url_file+file[0])
        print(file[1])
        download(url_file+file[0],file[1])
        if 'hwp' in file[1]:
            read_hwp_file(file[1])
        elif 'pdf' in file[1]:
            read_pdf_file(file[1])
        else:
            pass    # xls파일인 경우
    break

http://www.law.go.kr/LSW/flDownload.do?flSeq=25381477
law0126192016072500671KC_ancb.hwp
4    685       \x05HwpSummaryInformation


UnicodeDecodeError: 'utf-16-le' codec can't decode bytes in position 76-77: illegal UTF-16 surrogate

In [48]:
url_file = 'http://www.law.go.kr/LSW/'
for page in range(1,10):
    id_ls = get_url(page)
    file_ls = get_file(id_ls)
    
    for files in file_ls:
        for file in files:
            print(url_file+file[0])
            print(file[1])
            download(url_file+file[0],file[1])
            
            if 'hwp' in file[1]:
                read_hwp_file(file[1])
            elif 'pdf' in file[1]:
                read_pdf_file(file[1])
            else:
                pass    # xls파일인 경우

            
            
        break
    break

KeyboardInterrupt: 

In [26]:
pdf_file = urlopen("http://pythonscraping.com/pages/warandpeace/chapter1.pdf")
# pdf_file = open("out.pdf", "rb")                                      
contents = read_pdf_file(pdf_file)
print(contents)
pdf_file.close()

CHAPTER I

"Well, Prince, so Genoa and Lucca are now just family estates of 
theBuonapartes. But I warn you, if you don't tell me that this 
means war,if you still try to defend the infamies and horrors 
perpetrated bythat Antichrist- I really believe he is Antichrist- I will 
havenothing more to do with you and you are no longer my friend, 
no longermy 'faithful slave,' as you call yourself! But how do you 
do? I seeI have frightened you- sit down and tell me all the news."

It was in July, 1805, and the speaker was the well-known 
AnnaPavlovna Scherer, maid of honor and favorite of the 
Empress MaryaFedorovna. With these words she greeted Prince 
Vasili Kuragin, a manof high rank and importance, who was the 
first to arrive at herreception. Anna Pavlovna had had a cough for 
some days. She was, asshe said, suffering from la grippe; grippe 
being then a new word inSt. Petersburg, used only by the elite.

All her invitations without exception, written in French, 
anddelivered by a scar